In [ ]:
import os
import time
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import log_loss

from models_utils_skf import *

In [ ]:
def load_oof(mode = 'train'):
    src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/models/OOF_preds/'
    oof_preds = pd.DataFrame()
    files = sorted(os.listdir(src + '{}/'.format(mode)))
    print('Loading OOF preds:', files)
    for i in files:
        df_preds = pd.read_pickle('{}/{}/{}'.format(src, mode, i))
        oof_preds = pd.concat([oof_preds, df_preds], axis = 1)
    oof_preds.columns = ['{}_OOFpreds'.format(i) for i in range(oof_preds.shape[1])]
    return oof_preds

def transform(x):
    a = 0.165 / 0.37
    b =  (1 - 0.165) / (1 - 0.37)
    xt = a * x / (a * x + b * (1 - x))
    return xt

def predict_test_lgbm(test_preds, model_name):
    print('Predicting on test set with LightGBM.')
    sub_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/submissions/'
    sample_sub = pd.read_csv(sub_src + 'sample_submission.csv')
    sample_sub['is_duplicate'] = test_preds
    sample_sub.is_duplicate = sample_sub.is_duplicate.apply(transform)
    sample_sub.to_csv(sub_src + '{}.csv'.format(model_name), index = False)
    return

lgb_params = {
    'task' : 'train',
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : {'binary_logloss'},
    'learning_rate' : 0.03,
    'feature_fraction' : 0.51,
    'bagging_fraction': 0.9,
    'bagging_freq': 100,
    'num_leaves' : 255,
    'max_depth': 4,
    'min_data_in_leaf': 23,
    'subsample': 0.8,
    'colsample_bytree': 0.41,
    'silent': 1,
    'random_state': 1337,
    'verbose': 1,
    'nthread': 4,
}

xgb_params = {
    'seed': 1337,
    'colsample_bytree': 0.42,
    'silent': 1,
    'subsample': 0.85,
    'eta': 0.02,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 4,
    'min_child_weight': 20,
    'nthread': 4,
    }

In [ ]:
X_train = load_oof()
X_test = load_oof(mode = 'test')
y_train = pd.read_pickle('y_train.pkl')

X_train = X_train.iloc[:, 2:]
X_test = X_test.iloc[:, 2:]

In [ ]:
oof_test = lgb_foldrun_test(X_train, y_train, X_test, lgb_params, 'Stacking1Exp')

In [ ]:
oof_test

In [ ]:
predict_test_lgbm(oof_test, 'Stacking1Exp')